In [4]:
import os
import re
import numpy as np
import tifffile as tiff
import cv2

In [5]:
def max_projection_montage(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok = True)  # Ensure output folder exists
    
    # Regex to match files containing '-F4' to '-F11'
    pattern = re.compile(r'(.+)_([^-]+) - (.+)_(\w+_\w+)\.tif')
    
    projections = {"24h": [], "48h": [], "72h": []}
    
    for filename in os.listdir(input_folder):
        match = pattern.match(filename)
        print(match.group(0))
        if filename.endswith(".tif") and match:
            base_name, time, specific, name_pair = match.groups()
            input_path = os.path.join(input_folder, filename)
            
            with tiff.TiffFile(input_path) as tif_file:
                img = tif_file.asarray()
                metadata = tif_file.pages[0].tags
                resolution_tag = metadata.get('XResolution')
                pixel_size = 0.1883734  # Default pixel size in microns
                if resolution_tag:
                    resolution = resolution_tag.value
                    pixel_size = resolution[1] / resolution[0]
            
            if len(img.shape) == 4 and img.shape[1] == 4:  # (Z, C, H, W) format
                img = np.moveaxis(img, 1, 0)  # Convert to (C, Z, H, W)
                
                # Consider only Z slices 5-16 for projection
                img = img[:, 4:16, :, :]
                
                ch3_proj = np.max(img[2], axis = 0)
                ch2_proj = np.max(img[1], axis = 0)
                
                # Threshold and find top 3 brightest objects
                _, thresh = cv2.threshold(ch3_proj, 0, 255, cv2.THRESH_OTSU)
                contours, _ = cv2.findContours(thresh.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                objects = sorted(contours, key=cv2.contourArea, reverse=True)[:1]
                
                for idx, cnt in enumerate(objects):
                    x, y, w, h = cv2.boundingRect(cnt)
                    center_x, center_y = x + w // 2, y + h // 2
                    
                    half_size = 75  # 150x150 crop
                    x1, x2 = center_x - half_size, center_x + half_size
                    y1, y2 = center_y - half_size, center_y + half_size
                    
                    pad_x1, pad_x2, pad_y1, pad_y2 = 0, 0, 0, 0
                    if x1 < 0:
                        pad_x1 = abs(x1)
                        x1 = 0
                    if x2 > img.shape[3]:
                        pad_x2 = x2 - img.shape[3]
                        x2 = img.shape[3]
                    if y1 < 0:
                        pad_y1 = abs(y1)
                        y1 = 0
                    if y2 > img.shape[2]:
                        pad_y2 = y2 - img.shape[2]
                        y2 = img.shape[2]
                    
                    ch3_crop = img[2, :, y1:y2, x1:x2]
                    ch2_crop = img[1, :, y1:y2, x1:x2]
                    
                    ch3_crop = np.pad(ch3_crop, ((0, 0), (pad_y1, pad_y2), (pad_x1, pad_x2)), mode='constant', constant_values=0)
                    ch2_crop = np.pad(ch2_crop, ((0, 0), (pad_y1, pad_y2), (pad_x1, pad_x2)), mode='constant', constant_values=0)
                    
                    def safe_normalize(img):
                        if np.max(img) == np.min(img):
                            return np.zeros_like(img, dtype=np.uint8)
                        return cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                    
                    ch3_gray = safe_normalize(ch3_crop)
                    ch2_gray = safe_normalize(ch2_crop)
                    merged = cv2.merge((ch3_gray, ch2_gray, np.zeros_like(ch3_gray)))

                    # Create the merged image with ch3 in cyan and ch2 in grayscale
                    merged = cv2.merge((ch3_gray, ch3_gray, np.zeros_like(ch3_gray)))  # Cyan channel
                    grayscale = cv2.merge((ch2_gray, ch2_gray, ch2_gray))  # Grayscale channel
                    
                    # Combine the cyan and grayscale images using max blending to preserve both
                    final_merged = np.maximum(merged, grayscale)
                    
                    for key in projections:
                        if key in filename:
                            projections[key].append((final_merged, filename))
                            #projections[key].append((merged, filename))
                            break
    
    cols = 16
    rows_per_day = 3
    img_h, img_w, _ = 150, 150, 3
    montage = np.zeros((rows_per_day * 3 * img_h, cols * img_w, 3), dtype=np.uint8)
    
    for day_idx, key in enumerate(["24h", "48h", "72h"]):
        row_projections = projections[key]
        for idx, (img, filename) in enumerate(row_projections):
            row = (idx // 8) + (day_idx * 3)
            col = idx % 8 if "_IM_" in filename else (idx % 8) + 8
            
            y1, y2 = row * img_h, (row + 1) * img_h
            x1, x2 = col * img_w, (col + 1) * img_w
            
            montage[y1:y2, x1:x2] = img
           # cv2.putText(montage, filename, (x1 + 5, y1 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    
    output_path = os.path.join(output_folder, "montage.tif")
    with tiff.TiffWriter(output_path) as tif:
        tif.write(montage, metadata={'axes': 'YX', 'XResolution': (1, pixel_size), 'YResolution': (1, pixel_size)})
    
    print(f'Created montage: {output_path}')

In [6]:
input_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250204/01_Importer"
output_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250204/03_Galleries_V4"
max_projection_montage(input_folder, output_folder)

ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R04-C04-F8_D4_IM_CALM1_init.tif
ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R04-C04-F9_D4_IM_CALM1_init.tif
ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R04-C04-F10_D4_IM_CALM1_init.tif
ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R04-C04-F11_D4_IM_CALM1_init.tif
ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R04-C05-F0_D5_IM_CAMSAP1_init.tif
ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R02-C03-F3_B3_IM_KIF11_orig.tif
ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R02-C03-F4_B3_IM_KIF11_orig.tif
ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R02-C03-F5_B3_IM_KIF11_orig.tif
ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R02-C03-F6_B3_IM_KIF11_orig.tif
ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R02-C03-F7_B3_IM_KIF11_orig.tif
ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R02-C03-F8_B3_IM_KIF11_orig.tif
ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R02-C03-F9_B3_IM_KIF11_orig.tif
ID0080_TK_CID_RNAiScreen_63x_ReScan_72h - R02-C03-F10_B3_IM_KIF11_orig.tif
ID0080_TK_CID_RNAiScreen_63x_ReSc

KeyboardInterrupt: 